PLEASE NOTE: There is no reason to use this code.  As an admin, you can export all of the files!  I am onlyl including this for future records.  

In [6]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.options import Options
import time
import requests
from urllib.parse import urljoin
import os
import re
import logging

# Set up logging
logging.basicConfig(filename='download_errors.log', level=logging.ERROR)

# Browser options
options = Options()
options.add_argument("--start-maximized")
options.add_argument("--disable-extensions")
options.add_argument("--disable-gpu")

# Initialize the Edge driver
service = EdgeService(executable_path=r'C:\edgedriver\msedgedriver.exe')
driver = webdriver.Edge(service=service, options=options)

# URL you want to process
base_url = 'https://groups.io/g/Emco-CNC-Users/files'

print(f'Opening URL: {base_url}')
driver.get(base_url)

# Wait for manual login
print("Please log in manually. The browser window will remain open.")
input("Press Enter after you have logged in...")

# Continue with processing
def download_file(file_url):
    try:
        response = requests.get(file_url)
        file_name = file_url.split("/")[-1]
        with open(file_name, 'wb') as f:
            f.write(response.content)
        print(f'Downloaded: {file_name}')
    except Exception as e:
        logging.error(f"Failed to download {file_url}. Error: {e}")

def process_page(url):
    driver.get(url)
    time.sleep(5)  # Allow time for page to load

    page_source = driver.page_source
    links = re.findall(r'href=["\'](https?://[^\s\'"]*files[^\s\'"]*)["\']', page_source)
    
    for link in links:
        print(f'Found link: {link}')
        if link.endswith(('.pdf', '.doc', '.jpg', '.jpeg', '.pst')):
            print(f'Downloading {link}')
            download_file(link)
        else:
            print(f'Skipping {link}, not a valid file type')

# Start processing
process_page(base_url)

# Optionally, close the browser after processing
driver.quit()


Opening URL: https://groups.io/g/Emco-CNC-Users/files
Please log in manually. The browser window will remain open.
Found link: https://groups.io/g/Emco-CNC-Users/files
Skipping https://groups.io/g/Emco-CNC-Users/files, not a valid file type
Found link: https://groups.io/g/Emco-CNC-Users/files
Skipping https://groups.io/g/Emco-CNC-Users/files, not a valid file type
Found link: https://groups.io/g/Emco-CNC-Users/files?p=name,,,20,2,0,0
Skipping https://groups.io/g/Emco-CNC-Users/files?p=name,,,20,2,0,0, not a valid file type
Found link: https://groups.io/g/Emco-CNC-Users/files?p=type,,,20,1,0,0
Skipping https://groups.io/g/Emco-CNC-Users/files?p=type,,,20,1,0,0, not a valid file type
Found link: https://groups.io/g/Emco-CNC-Users/files?p=size,,,20,1,0,0
Skipping https://groups.io/g/Emco-CNC-Users/files?p=size,,,20,1,0,0, not a valid file type
Found link: https://groups.io/g/Emco-CNC-Users/files?p=created,,,20,1,0,0
Skipping https://groups.io/g/Emco-CNC-Users/files?p=created,,,20,1,0,0, n

In [6]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.options import Options
import time
import requests
from urllib.parse import urljoin, urlparse
import os
import re

# Browser options
options = Options()
options.add_argument("--start-maximized")
options.add_argument("--disable-extensions")
options.add_argument("--disable-gpu")

# Initialize the Edge driver
service = EdgeService(executable_path=r'C:\edgedriver\msedgedriver.exe')
driver = webdriver.Edge(service=service, options=options)

# URL you want to process
base_url = 'https://groups.io/g/Emco-CNC-Users/files'

print(f'Opening URL: {base_url}')
driver.get(base_url)

# Wait for manual login
print("Please log in manually. The browser window will remain open.")
input("Press Enter after you have logged in...")

# Initialize sets
visited_urls = set()
urls_to_visit = set([base_url])

def create_directory(path):
    if not os.path.exists(path):
        os.makedirs(path)

def save_html_page(url, html_content):
    parsed_url = urlparse(url)
    path = os.path.join('website_mirror', parsed_url.netloc, parsed_url.path.strip('/').replace('/', '_'))
    if not path.endswith('.html'):
        path += '.html'
    create_directory(os.path.dirname(path))
    with open(path, 'w', encoding='utf-8') as f:
        f.write(html_content)
    print(f'Saved HTML page: {path}')

def download_file(file_url):
    try:
        response = requests.get(file_url)
        file_name = file_url.split("/")[-1]
        parsed_url = urlparse(file_url)
        folder_path = os.path.join('website_mirror', parsed_url.netloc, os.path.dirname(parsed_url.path).strip('/').replace('/', '_'))
        create_directory(folder_path)
        file_path = os.path.join(folder_path, file_name)
        with open(file_path, 'wb') as f:
            f.write(response.content)
        print(f'Downloaded: {file_path}')
    except Exception as e:
        print(f"Failed to download {file_url}. Error: {e}")

def process_page(url):
    if url in visited_urls:
        return
    visited_urls.add(url)
    driver.get(url)
    time.sleep(5)  # Allow time for page to load

    page_source = driver.page_source
    save_html_page(url, page_source)
    
    links = re.findall(r'href=["\'](https?://[^\s\'"]*files[^\s\'"]*)["\']', page_source)
    
    for link in links:
        if link not in visited_urls:
            print(f'Found link: {link}')
            if link.endswith(('.pdf', '.doc', '.jpg', '.jpeg', '.pst')):
                print(f'Downloading {link}')
                download_file(link)
            else:
                urls_to_visit.add(link)
    
    # Recursively process new URLs
    while urls_to_visit:
        next_url = urls_to_visit.pop()
        if next_url not in visited_urls:
            process_page(next_url)

# Start processing
process_page(base_url)

# Optionally, close the browser after processing
driver.quit()


Opening URL: https://groups.io/g/Emco-CNC-Users/files
Please log in manually. The browser window will remain open.
Saved HTML page: website_mirror\groups.io\g_Emco-CNC-Users_files.html
Found link: https://groups.io/g/Emco-CNC-Users/files?p=name,,,20,2,0,0
Found link: https://groups.io/g/Emco-CNC-Users/files?p=type,,,20,1,0,0
Found link: https://groups.io/g/Emco-CNC-Users/files?p=size,,,20,1,0,0
Found link: https://groups.io/g/Emco-CNC-Users/files?p=created,,,20,1,0,0
Found link: https://groups.io/g/Emco-CNC-Users/files?p=updated,,,20,1,0,0
Found link: https://groups.io/g/Emco-CNC-Users/files/20160604090312.pdf
Downloaded: website_mirror\groups.io\g_Emco-CNC-Users_files\20160604090312.pdf
Found link: https://groups.io/g/Emco-CNC-Users/files/20160604090325.pdf
Downloaded: website_mirror\groups.io\g_Emco-CNC-Users_files\20160604090325.pdf
Found link: https://groups.io/g/Emco-CNC-Users/files/5PC%20-%20Conversion%20to%20MACH3
Found link: https://groups.io/g/Emco-CNC-Users/files/6%20Position

KeyboardInterrupt: 